In [1]:
from cProfile import label
from symbol import return_stmt

import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./UNSW_final_baseline_smote.csv')  # was './CICIDS-7-number-packets-to-flows_FINAL.csv'

In [ ]:
df.label.value_counts()

In [5]:
from sklearn.utils import resample

dfs = []

target_size = 1900  # a compromise between smallest and largest

for cls in df['label'].unique():
    class_df = df[df['label'] == cls]
    if len(class_df) > target_size:
        class_df = resample(class_df, replace=False, n_samples=target_size, random_state=42)
    else:
        class_df = resample(class_df, replace=True, n_samples=target_size, random_state=42)
    dfs.append(class_df)

balanced_df = pd.concat(dfs)


In [6]:
balanced_df.label.value_counts()

6    1900
7    1900
3    1900
4    1900
2    1900
1    1900
5    1900
8    1900
0    1900
9    1900
Name: label, dtype: int64

In [7]:
# Randomly shuffle the dataframe
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

In [8]:
df

,sbytes,state,sloss,dttl,sport,swin,tcprtt,dur,is_sm_ips_ports,ct_state_ttl,...,srcip,sjit,proto,spkts,dsport,flow_id,ackdat,trans_depth,res_bdy_len,label
0,203.821321,255.00000,199.099099,198.843844,255.000000,133.881381,199.609610,212.117117,225.007508,105.037538,...,255.000000,199.992492,126.989489,182.124625,255.000000,31.998850,199.737237,226.156156,227.687688,6
1,203.821321,255.00000,199.099099,198.843844,255.000000,133.881381,199.609610,212.117117,225.007508,105.037538,...,255.000000,199.992492,126.989489,182.124625,255.000000,31.881403,199.737237,226.156156,227.687688,6
2,203.821321,255.00000,199.099099,198.843844,255.000000,133.881381,199.609610,212.117117,225.007508,105.037538,...,255.000000,199.992492,126.989489,182.124625,255.000000,31.940636,199.737237,226.156156,227.687688,6
3,203.821321,255.00000,199.099099,198.843844,255.000000,133.881381,199.609610,212.117117,225.007508,105.037538,...,255.000000,199.992492,126.989489,182.124625,255.000000,31.905728,199.737237,226.156156,227.687688,6
4,203.821321,255.00000,199.099099,198.843844,255.000000,133.881381,199.609610,212.117117,225.007508,105.037538,...,255.000000,199.992492,126.989489,182.124625,255.000000,31.974109,199.737237,226.156156,227.687688,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28597,134.190606,50.03003,108.483483,240.322823,143.861806,160.810811,243.910325,241.655464,0.000000,145.750751,...,98.422154,234.775336,12.379880,194.356868,196.418919,157.749951,245.136251,0.000000,0.000000,9
28598,128.759131,50.03003,108.483483,240.322823,168.246676,160.810811,245.809966,193.019779,0.000000,145.750751,...,112.018667,232.291143,12.379880,161.246024,196.418919,167.759417,240.660915,252.702703,254.723231,9
28599,139.030907,50.03003,108.483483,240.322823,170.055414,160.810811,238.716286,241.284489,0.000000,145.750751,...,114.724035,232.335450,12.379880,219.820492,196.418919,173.413233,240.276771,0.000000,0.000000,9
28600,127.657727,50.03003,108.483483,240.322823,108.508705,160.810811,250.920636,242.156163,0.000000,145.750751,...,76.419762,238.068411,12.379880,159.989015,196.418919,136.609468,251.695015,0.000000,0.000000,9


In [10]:
count=0
ims = []

image_count = 0

label_count = {i: 0 for i in range(0,10)}


for i in range(0, len(balanced_df)):  
    count=count+1
    if count<=60: 
        im=balanced_df.iloc[i].drop('label').values
        label_count[balanced_df.iloc[i]['label']] += 1
        ims=np.append(ims,im)
    else:
        max_value = max(label_count.values())  # maximum value
        max_keys = [k for k, v in label_count.items() if v == max_value] # getting all keys containing the `maximum`
        
        this_label = max_keys[0]
        image_path = f"test_shuffled/{this_label}/"
        
        if not os.path.exists(image_path):
            os.makedirs(image_path)
        
        image_count = image_count + 1
        ims=np.array(ims).reshape(20,20,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []
        label_count = {i: 0 for i in range(0,10)}

In [11]:
#resize the images 224*224 for better CNN training
def get_224(folder,dstdir):
    imgfilepaths=[]
    for root,dirs,imgs in os.walk(folder):
        for thisimg in imgs:
            thisimg_path=os.path.join(root,thisimg)
            imgfilepaths.append(thisimg_path)
    for thisimg_path in imgfilepaths:
        dir_name,filename=os.path.split(thisimg_path)
        dir_name=dir_name.replace(folder,dstdir)
        new_file_path=os.path.join(dir_name,filename)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        img=cv2.imread(thisimg_path)
        img=cv2.resize(img,(224,224))
        cv2.imwrite(new_file_path,img)
    print('Finish resizing'.format(folder=folder))

In [12]:
DATA_DIR2_224='./test_shuffled_224/'
get_224(folder='./test_shuffled/',dstdir=DATA_DIR2_224)

Finish resizing


In [13]:
label_count

{0: 5, 1: 4, 2: 2, 3: 3, 4: 3, 5: 1, 6: 2, 7: 3, 8: 3, 9: 3}